##### More info about the data: https://www.isi.edu/~lerman/downloads/digg2009.html

In [2]:
import numpy as np

# votes = np.fromregex("digg2009/digg_votes.csv", '"(\d+)","(\d+)","(\d+)"', np.uint32)
# friends = np.fromregex("digg2009/digg_friends.csv", '"(\d+)","(\d+)","(\d+)","(\d+)"', np.uint32)

In [3]:
# save these as npy files so we don't need to keep downloading the original csvs
# still big but saves about half the space
np.save("digg_votes", votes)
np.save("digg_friends", friends)

votes = np.load("digg_votes.npy")
friends = np.load("digg_friends.npy")

NameError: name 'votes' is not defined

#### Digg_Votes
The `votes` table has the following columns:
 - 0: Unix time stamp of the vote
 - 1: Unique id of the voter
 - 2: Unique id of the story

#### Digg_Friends
The `friends` table has the following columns:
 - 0: 1 if the friend follows them back, 0 if not (F4F 🥴)
 - 1: Unix Timestamp of when they added the friend
 - 2: Unique id of the user
 - 3: Unique id of the friend

In [5]:

def print_summary(table):
    print(table.shape[0], "rows, ", table.shape[1], "cols, ", np.ceil(table.nbytes/1048576), "MB, ",)

print_summary(votes)
print_summary(friends)

votes_for_story_1 = votes[votes[:,2] == 1] # Select storyId = 1
print_summary(votes_for_story_1)

# Select friendId = 336224
# I'm using numpy masks here to select when either column 2 `or` (|) 3 are = 336224.
friends_with_user_336224 = friends[(friends[:,2] == 336224) | (friends[:,3] == 336224)]
print_summary(friends_with_user_336224)


3018197 rows,  3 cols,  35.0 MB, 
1731658 rows,  4 cols,  27.0 MB, 
583 rows,  3 cols,  1.0 MB, 
9 rows,  4 cols,  1.0 MB, 


#### start of SI model stuff pls ignore

In [4]:
import pandas as pd

votes = pd.read_csv("digg2009/digg_votes.csv", header = None)
friends = pd.read_csv("digg2009/digg_friends.csv", header = None)

print(votes.head())
print(friends.head())

            0     1  2
0  1246573330   318  1
1  1246603391   399  1
2  1246573479  1749  1
3  1246599553  1964  1
4  1246568035  4362  1
   0           1       2       3
0  1  1214799565  336224  266641
1  0  1230047012  336224  215086
2  1  1194104160  336224  151094
3  0  1199180573  336224   59378
4  0  1196465462  336224    8547


In [6]:
num_voter = votes.groupby(2).count()
print(num_voter.head())
max_votes_idx = num_voter.idxmax()
print()
print(max_votes_idx)

max_votes = num_voter.max()
print()
print(max_votes)

     0    1
2          
1  583  583
2  391  391
3  534  534
4  326  326
5  691  691

0    714
1    714
dtype: int64

0    24099
1    24099
dtype: int64


In [12]:
# id of story that has the max votes 
print(num_voter.loc[714])
print(votes.loc[714,2])

0    24099
1    24099
Name: 714, dtype: int64
2


In [29]:
uniq_voters = votes.groupby(1).count()
print(uniq_voters)

v = votes[1].max()
print("num voters:", v)

          0    2
1               
2         4    4
5        17   17
6        74   74
7        26   26
8       153  153
...     ...  ...
336214   16   16
336215   30   30
336216   87   87
336217    1    1
336224    3    3

[139409 rows x 2 columns]
num voters: 336224


In [28]:
f = friends[2].max()
print("num users:", f)

         0   1   3
2                 
5        7   7   7
7        1   1   1
8        3   3   3
10      19  19  19
11       1   1   1
...     ..  ..  ..
336214  10  10  10
336215   2   2   2
336216  19  19  19
336217   4   4   4
336224   5   5   5

[71367 rows x 3 columns]
num users 336224


In [36]:
#calculating avg num of friends 

uniq_fans = friends.groupby(2).count()
print(uniq_fans[3])



2
5          7
7          1
8          3
10        19
11         1
          ..
336214    10
336215     2
336216    19
336217     4
336224     5
Name: 3, Length: 71367, dtype: int64


In [43]:
total = uniq_fans[3].sum(axis = 0)
#R0 is the contact rate (transmission) so each user has on avg 5.15 fans 
print("R0 = ", total/f)

R0 =  5.15031050728086
